In [4]:
# !pip install langchain

In [5]:
# !pip install langchain-community

In [7]:
# !pip install langchain-groq

In [3]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq

In [7]:
import os

# Set the API key environment variable
os.environ['GROQ_API_KEY'] = grisha_llama_key


In [8]:
llm = ChatGroq(temperature=0, model_name="llama-3.3-70b-versatile")

In [30]:
response=llm.invoke("who is the first person who walk in moon?")
print(response)

content='The first person to walk on the Moon was Neil Armstrong. He stepped out of the lunar module Eagle and onto the Moon\'s surface on July 20, 1969, during the Apollo 11 mission. Armstrong famously declared, "That\'s one small step for man, one giant leap for mankind," as he became the first human to set foot on the Moon.' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 75, 'prompt_tokens': 45, 'total_tokens': 120, 'completion_time': 0.272727273, 'prompt_time': 0.00232816, 'queue_time': 0.20231019, 'total_time': 0.275055433}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_8dd9fca28c', 'finish_reason': 'stop', 'logprobs': None} id='run-ed177ebc-dde3-4938-8f77-48e7bbb341c1-0' usage_metadata={'input_tokens': 45, 'output_tokens': 75, 'total_tokens': 120}


In [33]:
# !pip install chromadb
import chromadb

In [29]:
# !pip install --upgrade groq


In [34]:
from typing import Collection
client=chromadb.Client()
Collection=client.create_collection("grisha_collection")


In [36]:
Collection.add(
    documents=[
        "This document is about New York",
        "This document is about Delhi"
    ],
    ids = ['id1', 'id2']
)

/root/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:00<00:00, 84.8MiB/s]


In [38]:
all_docs = Collection.get()
all_docs

{'ids': ['id1', 'id2'],
 'embeddings': None,
 'documents': ['This document is about New York',
  'This document is about Delhi'],
 'uris': None,
 'data': None,
 'metadatas': [None, None],
 'included': [<IncludeEnum.documents: 'documents'>,
  <IncludeEnum.metadatas: 'metadatas'>]}

In [41]:
results = Collection.query(
    query_texts=['Query is about Pizza'],
    n_results=2
)
results

{'ids': [['id1', 'id2']],
 'embeddings': None,
 'documents': [['This document is about New York',
   'This document is about Delhi']],
 'uris': None,
 'data': None,
 'metadatas': [[None, None]],
 'distances': [[1.6637976169586182, 1.7067852020263672]],
 'included': [<IncludeEnum.distances: 'distances'>,
  <IncludeEnum.documents: 'documents'>,
  <IncludeEnum.metadatas: 'metadatas'>]}

In [42]:
Collection.delete(ids=all_docs['ids'])
Collection.get()

{'ids': [],
 'embeddings': None,
 'documents': [],
 'uris': None,
 'data': None,
 'metadatas': [],
 'included': [<IncludeEnum.documents: 'documents'>,
  <IncludeEnum.metadatas: 'metadatas'>]}

In [44]:

Collection.add(
    documents=[
        "This document is about New York",
        "This document is about Delhi"
    ],
    ids=["id3", "id4"],
    metadatas=[
        {"url": "https://en.wikipedia.org/wiki/New_York_City"},
        {"url": "https://en.wikipedia.org/wiki/New_Delhi"}
    ]
)

In [46]:

results = Collection.query(
    query_texts=["Query is about Chhole Bhature"],
    n_results=2
)
results

{'ids': [['id4', 'id3']],
 'embeddings': None,
 'documents': [['This document is about Delhi',
   'This document is about New York']],
 'uris': None,
 'data': None,
 'metadatas': [[{'url': 'https://en.wikipedia.org/wiki/New_Delhi'},
   {'url': 'https://en.wikipedia.org/wiki/New_York_City'}]],
 'distances': [[1.5588479042053223, 1.8114912509918213]],
 'included': [<IncludeEnum.distances: 'distances'>,
  <IncludeEnum.documents: 'documents'>,
  <IncludeEnum.metadatas: 'metadatas'>]}

In [24]:

from langchain_community.document_loaders import WebBaseLoader

In [25]:
loader = WebBaseLoader("https://careers.nike.com/analyst-supply-chain-trading-company/job/R-54896")
page_data = loader.load().pop().page_content
print(page_data)






















Analyst, Supply Chain Trading Company










































Skip to main content
Open Virtual Assistant










Home


Career Areas


Total Rewards


Life@Nike


Purpose










Language





Select a Language

  Deutsch  
  English  
  Español (España)  
  Español (América Latina)  
  Français  
  Italiano  
  Nederlands  
  Polski  
  Tiếng Việt  
  Türkçe  
  简体中文  
  繁體中文  
  עִברִית  
  한국어  
  日本語  








Careers


















Close Menu







Careers






Chat






                                Home
                            



                                Career Areas
                            



                                Total Rewards
                            



                                Life@Nike
                            



                                Purpose
                            










Jordan Careers







Converse Careers










Language











Menu



Return to Previ

In [27]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):
        """
)

chain_extract = prompt_extract | llm
res = chain_extract.invoke(input={'page_data':page_data})
print(res.content)

```json
{
  "role": "Analyst, Supply Chain Trading Company",
  "experience": "Minimum 5 years of experience in Supply Chain and/or Logistics Operation",
  "skills": [
    "SAP across Sales & Distribution (SD) and Material Management (MM)",
    "Order Management and Vendor management",
    "Demand / supply planning and shipping knowledge",
    "Microsoft Office software suites and report generation",
    "Smartsheet, Tableau and/or Data Bricks",
    "Process automation and related capabilities (eg RPA/ Microsoft Power Automate)"
  ],
  "description": "Supports direct materials buy-sell operations, partnering Nike global teams to manage external vendors to ensure timely delivery of raw materials to finished goods factories. Responsible for the timely availability of raw materials from vendors to Finished Goods factories through effective and efficient order/delivery management while maintaining high operational standards and compliance."
}
```


In [31]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

{'role': 'Analyst, Supply Chain Trading Company',
 'experience': 'Minimum 5 years of experience in Supply Chain and/or Logistics Operation',
 'skills': ['SAP across Sales & Distribution (SD) and Material Management (MM)',
  'Order Management and Vendor management',
  'Demand / supply planning and shipping knowledge',
  'Microsoft Office software suites and report generation',
  'Smartsheet, Tableau and/or Data Bricks',
  'Process automation and related capabilities (eg RPA/ Microsoft Power Automate)'],
 'description': 'Supports direct materials buy-sell operations, partnering Nike global teams to manage external vendors to ensure timely delivery of raw materials to finished goods factories. Responsible for the timely availability of raw materials from vendors to Finished Goods factories through effective and efficient order/delivery management while maintaining high operational standards and compliance.'}

In [12]:
# !pip install trafilatura


In [28]:
# from langchain_core.prompts import PromptTemplate

# prompt_extract = PromptTemplate.from_template(
#         """
#         ### SCRAPED TEXT FROM WEBSITE:
#         {page_data}
#         ### INSTRUCTION:
#         The scraped text is from the career's page of a website.
#         Your job is to extract the job postings and return them in JSON format containing the
#         following keys: `role`, `experience`, `skills` and `description`.
#         Only return the valid JSON.
#         ### VALID JSON (NO PREAMBLE):
#         """
# )

# import trafilatura

# url = "https://careers.nike.com/analyst-supply-chain-trading-company/job/R-54896"
# downloaded = trafilatura.fetch_url(url)

# if downloaded:
#     text = trafilatura.extract(downloaded)
#     chain_extract = prompt_extract | llm
#     res = chain_extract.invoke(input={'page_data':text})
#     print(res.content)
#     # print(text)  # Clean job description
# else:
#     print("Failed to fetch webpage.")
# # downloaded


In [30]:
# !pip install selenium webdriver-manager


In [32]:
import pandas as pd

df = pd.read_csv("/content/my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [33]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [35]:
links=collection.query(query_texts=["Expericence in python and R","little bit knowlegde of django"],n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/python-portfolio'},
  {'links': 'https://example.com/ml-python-portfolio'}]]

In [37]:
job = json_res
job['skills']

['SAP across Sales & Distribution (SD) and Material Management (MM)',
 'Order Management and Vendor management',
 'Demand / supply planning and shipping knowledge',
 'Microsoft Office software suites and report generation',
 'Smartsheet, Tableau and/or Data Bricks',
 'Process automation and related capabilities (eg RPA/ Microsoft Power Automate)']

In [38]:
job

{'role': 'Analyst, Supply Chain Trading Company',
 'experience': 'Minimum 5 years of experience in Supply Chain and/or Logistics Operation',
 'skills': ['SAP across Sales & Distribution (SD) and Material Management (MM)',
  'Order Management and Vendor management',
  'Demand / supply planning and shipping knowledge',
  'Microsoft Office software suites and report generation',
  'Smartsheet, Tableau and/or Data Bricks',
  'Process automation and related capabilities (eg RPA/ Microsoft Power Automate)'],
 'description': 'Supports direct materials buy-sell operations, partnering Nike global teams to manage external vendors to ensure timely delivery of raw materials to finished goods factories. Responsible for the timely availability of raw materials from vendors to Finished Goods factories through effective and efficient order/delivery management while maintaining high operational standards and compliance.'}

In [40]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}

        ### INSTRUCTION:
        You are Viren, a business development executive at AtliQ. AtliQ is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools.
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability,
        process optimization, cost reduction, and heightened overall efficiency.
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of AtliQ
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Atliq's portfolio: {link_list}
        Remember you are Mohan, BDE at AtliQ.
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):

        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Enhancing Supply Chain Efficiency with AtliQ's Automated Solutions

Dear Hiring Manager,

I came across the job description for an Analyst, Supply Chain Trading Company, and I was impressed by the role's focus on optimizing supply chain operations. As a Business Development Executive at AtliQ, I believe our company's expertise in AI and software consulting can help streamline your supply chain processes, ensuring timely delivery of raw materials and finished goods.

At AtliQ, we have a proven track record of empowering enterprises with tailored solutions that foster scalability, process optimization, cost reduction, and heightened overall efficiency. Our team can help you leverage SAP across Sales & Distribution (SD) and Material Management (MM), as well as implement process automation and related capabilities like RPA/Microsoft Power Automate. We can also assist with order management, vendor management, demand/supply planning, and shipping knowledge, all while maintaining hig